In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os
import folium


In [3]:
def print_metadata(image_path):
    print(f"Extracting metadata from {image_path}")
    image = Image.open(image_path)
    metadata = image._getexif()
    if metadata:
        for tag_id, value in metadata.items():
            tag_name = TAGS.get(tag_id, tag_id)
            print(f"{tag_name}: {value}")
    else:
        print("No EXIF metadata found.")

def print_gps_info(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data:
        for tag_id in exif_data:
            tag_name = TAGS.get(tag_id, tag_id)
            if tag_name == "GPSInfo":
                gps_data = exif_data[tag_id]
                for gps_tag in gps_data:
                    gps_tag_name = GPSTAGS.get(gps_tag, gps_tag)
                    print(f"{gps_tag_name}: {gps_data[gps_tag]}")
    else:
        print("No EXIF metadata found.")


cwd = os.getcwd()
folders = os.listdir(cwd)
folders = [os.path.join(cwd, folder) for folder in folders if os.path.isdir(os.path.join(cwd, folder))]
print (folders)
folder = folders[0]
print(folder)
files = os.listdir(folder)

print(files)
for file in files:
    if file.endswith(".JPG"):
        print_metadata(os.path.join(folder, file))
        print_gps_info(os.path.join(folder, file))
        print()

['d:\\Uni\\Year 3\\CV\\Assignment\\Jer\\General', 'd:\\Uni\\Year 3\\CV\\Assignment\\Jer\\Organic', 'd:\\Uni\\Year 3\\CV\\Assignment\\Jer\\Recycling']
d:\Uni\Year 3\CV\Assignment\Jer\General
['IMG_4284.JPG', 'IMG_4285.JPG', 'IMG_4286.JPG', 'IMG_4287.JPG', 'IMG_4288.JPG', 'IMG_4289.JPG', 'IMG_4290.JPG', 'IMG_4292.JPG', 'IMG_4293.JPG', 'IMG_4294.JPG', 'IMG_4295.JPG', 'IMG_4296(e).JPG', 'IMG_4297.JPG', 'IMG_4298(e).JPG', 'IMG_4299.JPG', 'IMG_4300.JPG', 'IMG_4301.JPG', 'IMG_4302.JPG', 'IMG_4303.JPG', 'IMG_4304.JPG', 'IMG_4305.JPG', 'IMG_4306.JPG', 'IMG_4307.JPG', 'IMG_4308(e).JPG', 'IMG_4309.JPG', 'IMG_4310.JPG', 'IMG_4311.JPG', 'IMG_4312(e).JPG', 'IMG_4313(e).JPG', 'IMG_4314.JPG', 'IMG_4315.JPG', 'IMG_4316.JPG', 'IMG_4317.JPG', 'IMG_4318.JPG', 'IMG_4319.JPG', 'IMG_4320.JPG', 'IMG_4321.JPG', 'IMG_4322.JPG', 'IMG_4323.JPG', 'IMG_4324.JPG', 'IMG_4325.JPG', 'IMG_4326.JPG', 'IMG_4327.JPG', 'IMG_4328.JPG', 'IMG_4329.JPG', 'IMG_4330.JPG']
Extracting metadata from d:\Uni\Year 3\CV\Assignment\Jer\G

In [17]:
def get_geotags(image_path):
    image = Image.open(image_path)
    metadata = image._getexif()
    if not metadata:
        return None

    gps_info = metadata.get(34853)  # GPSInfo tag ID
    if gps_info:
        geotags = {GPSTAGS.get(tag, tag): value for tag, value in gps_info.items()}
        return geotags
    return None

def convert_to_degrees(value):
    d, m, s = value
    return d + (m / 60.0) + (s / 3600.0)

def get_coordinates(geotags):
    lat = convert_to_degrees(geotags["GPSLatitude"])
    if geotags["GPSLatitudeRef"] == "S":
        lat = -lat

    lon = convert_to_degrees(geotags["GPSLongitude"])
    if geotags["GPSLongitudeRef"] == "W":
        lon = -lon

    return lat, lon

def find_center(coordinates):
    # Find the center of a set of coordinates
    lats = [coord[0] for coord in coordinates]
    lons = [coord[1] for coord in coordinates]
    return sum(lats) / len(coordinates), sum(lons) / len(coordinates)

def create_map(markers, classes, class2col, zoom=10):
    map = folium.Map(location=find_center(markers), zoom_start=zoom)
    for coord, cls in zip(markers, classes):
        folium.Marker(
            location=coord,
            popup=cls,
            icon=folium.Icon(color=class2col[cls])
        ).add_to(map)
    return map

# Create a map with markers for each image
coordinates = []
classes = []

class2col = {"General": "blue", "Organic": "red", "Recycling": "green"}

for folder in folders:
    files = os.listdir(folder)
    for file in files:
        if file.endswith(".JPG"):
            geotags = get_geotags(os.path.join(folder, file))
            if geotags:
                coordinates.append(get_coordinates(geotags))
                classes.append(folder.split("\\")[-1])

map = create_map(coordinates, classes, class2col, 15)

map
